In [109]:
import numpy as np
import pandas as pd
import pymap3d
from numpy.linalg import inv
import warnings
warnings.filterwarnings('ignore')

In [110]:
class GNSS_filter:
    def __init__(self, row):
        dt = 5e-2

        self.Nx = 6  # number of states: pos, vel
        self.Nu = 3  # number of input var: vel

        self.X = np.zeros(
            [self.Nx, 1]
        )  # error states: pos_xyz, v_ned, eul, bias_acc, bias_gyro

        # Initial antenna location
        pos_xyz = np.array((row["x"], row["y"], row["z"])).reshape(3, 1)
        self.X[0:3] = pos_xyz
        # Initial antenna velocity
        rot, v_xyz = self.rot_vel(row)

        self.X[3:6] = v_xyz

        # Process model

        # State transition matrix
        self.F = np.identity(self.Nx)

        self.F[0:3, 3:6] = np.identity(3) * dt

        self.P = np.identity(self.Nx)  # Set initial values to 0.25?

        # Process noise matrix
        self.gnss_pos_psd = 3.125e-5
        self.vel_psd = 0.0025

        self.Q = np.zeros([self.Nx, self.Nx])

        self.updateQ(dt)

        # Position measurement noise
        self.R_position = np.identity(3)
        self.R_position[0, 0] = row.sdx**2
        self.R_position[1, 1] = row.sdy**2
        self.R_position[2, 2] = row.sdz**2

        self.R_position[0, 1] = np.sign(row.sdxy) * row.sdxy**2
        self.R_position[0, 2] = np.sign(row.sdxz) * row.sdxz**2
        self.R_position[1, 2] = np.sign(row.sdyz) * row.sdyz**2

        self.R_position[1, 0] = self.R_position[0, 1]
        self.R_position[2, 0] = self.R_position[0, 2]
        self.R_position[2, 1] = self.R_position[1, 2]

        # Velocity measurement noise
        self.R_velocity = np.identity(3) * 4e-8

        self.R_velocity[0, 1] = 1.5e-9
        self.R_velocity[0, 2] = 1.5e-9
        self.R_velocity[1, 2] = 1.5e-9

        self.R_velocity[1, 0] = self.R_velocity[0, 1]
        self.R_velocity[2, 0] = self.R_velocity[0, 2]
        self.R_velocity[2, 1] = self.R_velocity[1, 2]

        if "v_sde" in row:
            self.R_velocity[0, 0] = row.v_sde**2
            self.R_velocity[1, 1] = row.v_sdn**2
            self.R_velocity[2, 2] = row.v_sdu**2

            self.R_velocity[0, 1] = row.v_sden
            self.R_velocity[0, 2] = row.v_sdeu
            self.R_velocity[1, 2] = row.v_sdnu

            self.R_velocity[1, 0] = self.R_velocity[0, 1]
            self.R_velocity[2, 0] = self.R_velocity[0, 2]
            self.R_velocity[2, 1] = self.R_velocity[1, 2]

        self.R_velocity = rot @ self.R_velocity @ rot.transpose()

    def rot_vel(self, row):
        """
        ------------------- Rotate ENU velocity into ECEF velocity --------------------------------
                   dX = |  -sg   -sa*cg  ca*cg | | de |        de = |  -sg       cg      0 | | dX |
                   dY = |   cg   -sa*sg  ca*sg | | dn |  and   dn = |-sa*cg  -sa*sg     ca | | dY |
                   dZ = |    0    ca     sa    | | du |        du = | ca*cg   ca*sg     sa | | dZ |
        -------------------------------------------------------------------------------------------
        """
        v_enu = np.array((row["vel_e"], row["vel_n"], row["vel_u"])).reshape(3, 1)
        lat, lon, alt = pymap3d.ecef2geodetic(self.X[0], self.X[1], self.X[2])
        lat = np.deg2rad(lat)
        lam = np.deg2rad(lon)

        ca = np.cos(lat)
        sa = np.sin(lat)

        cg = np.cos(lam)
        sg = np.sin(lam)

        rot = np.zeros((3, 3))
        rot[0, 0] = -sg
        rot[0, 1] = -sa * cg
        rot[0, 2] = ca * cg
        rot[1, 0] = cg
        rot[1, 1] = -sa * sg
        rot[1, 2] = ca * sg
        rot[2, 0] = 0
        rot[2, 1] = ca
        rot[2, 2] = sa
        v_xyz = rot @ v_enu

        return rot, v_xyz

    def updateQ(self, dt):
        # Position estimation noise
        # Initial Q values from Chadwell code 3.125d-5 3.125d-5 3.125d-5 0.0025 0.0025 0.0025, assumes white noise of 2.5 cm over a second
        self.Q[0:3, 0:3] = np.identity(3) * self.gnss_pos_psd * dt

        # Velocity estimation noise (acc psd)
        self.Q[3:6, 3:6] = np.identity(3) * self.vel_psd

    def predict(self, dt):  # w is the angular rate vector
        self.F[0:3, 3:6] = np.identity(3) * dt

        self.updateQ(dt)

        self.X = self.F @ self.X
        self.P = self.F @ self.P @ self.F.transpose() + self.Q

    def updateVel_cov(self, row):
        rot, v_xyz = self.rot_vel(row)

        self.R_velocity[0, 0] = row.v_sde**2
        self.R_velocity[1, 1] = row.v_sdn**2
        self.R_velocity[2, 2] = row.v_sdu**2

        self.R_velocity[0, 1] = row.v_sden
        self.R_velocity[0, 2] = row.v_sdeu
        self.R_velocity[1, 2] = row.v_sdnu

        self.R_velocity[1, 0] = self.R_velocity[0, 1]
        self.R_velocity[2, 0] = self.R_velocity[0, 2]
        self.R_velocity[2, 1] = self.R_velocity[1, 2]
        self.R_velocity = rot @ self.R_velocity @ rot.transpose()

    def updatePosition(self, row):
        # 		pos_ned = row[['pos_n','pos_e','pos_d']].values.reshape(3,1)
        pos_xyz = np.array((row["x"], row["y"], row["z"])).reshape(3, 1)

        H = np.zeros([3, self.Nx])
        H[0:3, 0:3] = np.identity(3)

        self.R_position[0, 0] = row.sdx**2
        self.R_position[1, 1] = row.sdy**2
        self.R_position[2, 2] = row.sdz**2

        self.R_position[0, 1] = np.sign(row.sdxy) * row.sdxy**2
        self.R_position[0, 2] = np.sign(row.sdxz) * row.sdxz**2
        self.R_position[1, 2] = np.sign(row.sdyz) * row.sdyz**2

        self.R_position[1, 0] = self.R_position[0, 1]
        self.R_position[2, 0] = self.R_position[0, 2]
        self.R_position[2, 1] = self.R_position[1, 2]

        y = pos_xyz - H @ self.X
        S = H @ self.P @ H.transpose() + self.R_position
        K = (self.P @ H.transpose()) @ inv(S)
        self.X = self.X + K @ y

        I = np.identity(self.Nx)
        self.P = (I - K @ H) @ self.P

    def updateVelocity(self, row):
        # 		v_ned = row[['vel_n', 'vel_e', 'vel_d']].values.reshape(3,1)
        rot, v_xyz = self.rot_vel(row)

        H = np.zeros([3, self.Nx])
        H[0:3, 3:6] = np.identity(3)

        y = v_xyz - H @ self.X
        S = H @ self.P @ H.transpose() + self.R_velocity
        K = (self.P @ H.transpose()) @ inv(S)
        self.X = self.X + K @ y

        I = np.identity(self.Nx)
        self.P = (I - K @ H) @ self.P

    def get_states(self):
        return {
            "x": np.ndarray.item(self.X[0]),
            "y": np.ndarray.item(self.X[1]),
            "z": np.ndarray.item(self.X[2]),
            "vx": np.ndarray.item(self.X[3]),
            "vy": np.ndarray.item(self.X[4]),
            "vz": np.ndarray.item(self.X[5]),
        }

    def return_states(self):
        return self.X

    def return_covariance(self):
        return self.P

    def rts_smoother(self, Xs, Ps):
        # Rauch, Tongue, and Striebel smoother

        n, dim_x, _ = Xs.shape

        # smoother gain
        K = np.zeros((n, dim_x, dim_x))
        x, P, Pp = Xs.copy(), Ps.copy(), Ps.copy()

        i = 0
        for k in range(n - 2, -1, -1):
            Pp[k] = self.F @ P[k] @ self.F.T + self.Q  # predicted covariance

            K[k] = P[k] @ self.F.T @ inv(Pp[k])
            x[k] += K[k] @ (x[k + 1] - (self.F @ x[k]))
            P[k] += K[k] @ (P[k + 1] - Pp[k]) @ K[k].T
            i += 1
            print(str(i + 1) + "/" + str(n), end="\r")
        print("")

        return x, P, K, Pp


In [111]:
def run_filter_simulation(df):
    import time

    init = True

    results = []
    Xs = np.zeros([len(df), 6, 1])
    Ps = np.zeros([len(df), 6, 6])

    all_results = pd.DataFrame()

    # Process data through forward Kalman filter
    for i in range(len(df)):
        row = df[i]
        print(str(i + 1) + "/" + str(len(df)), end="\r")

        if init:
            gnss_kf = GNSS_filter(row)
            init = False
            last_time = row.dts
        else:
            dt = abs(
                row.dts - last_time
            )  # This helps to stabilize the solution, abs ensures reverse filtering works.
            gnss_kf.predict(dt)
            last_time = row.dts

            # New velocity standard deviations
            if "v_sde" in row.dtype.names:
                if not np.isnan(row["v_sde"]):
                    gnss_kf.updateVel_cov(row)
            # new GNSS measurement
            if not np.isnan(row["x"]):
                gnss_kf.updatePosition(row)
            # new velocity measurement
            if not np.isnan(row["vel_e"]):
                gnss_kf.updateVelocity(row)

        X = gnss_kf.return_states()
        P = gnss_kf.return_covariance()
        Xs[i] = X
        Ps[i] = P

    print("")
    print("Smoothing results")
    x, P, K, Pp = gnss_kf.rts_smoother(Xs, Ps)

    return x, P, K, Pp


In [112]:
from gnatss.loaders import read_novatel_L1_data_files

In [113]:
df_rph = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSPVAA.dat'], data_format="INSPVAA")

In [114]:
df_std = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSSTDEVA.dat'], data_format="INSSTDEVA")
 

In [115]:
df_std = df_std.drop_duplicates(subset="time", keep="first")
df_std

,Week,Seconds,lat std,lon std,alt std,ant_n std,ant_e std,ant_u std,roll_roll,pitch_pitch,heading_heading,Ext sol stat,Time Since Update,time
0,2220,400732.75,0.5002,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0,7.122935e+08
1,2220,400732.80,0.5002,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0,7.122935e+08
2,2220,400732.85,0.5002,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0,7.122935e+08
3,2220,400732.90,0.5002,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0,7.122935e+08
4,2220,400732.95,0.5002,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0,7.122935e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865303,2220,469048.15,0.3748,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0,7.123618e+08
3865304,2220,469048.20,0.3748,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0,7.123618e+08
3865305,2220,469048.25,0.3748,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0,7.123618e+08
3865306,2220,469048.30,0.3748,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0,7.123618e+08


In [116]:
vel_std_df = df_rph.merge(df_std, on="time")
vel_std_df

,Week_x,Seconds_x,lat,lon,alt,ant_n,ant_e,ant_u,roll,pitch,...,lon std,alt std,ant_n std,ant_e std,ant_u std,roll_roll,pitch_pitch,heading_heading,Ext sol stat,Time Since Update
0,2220,400732.75,45.302281,-124.965970,-28.3449,-0.6858,0.1014,0.2413,-3.550174,1.675695,...,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0
1,2220,400732.80,45.302281,-124.965970,-28.3340,-0.6380,0.0788,0.2734,-3.540661,1.372733,...,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0
2,2220,400732.85,45.302281,-124.965970,-28.3225,-0.5833,0.0534,0.2926,-3.183020,1.125801,...,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0
3,2220,400732.90,45.302280,-124.965970,-28.3104,-0.5440,0.0405,0.2950,-2.585131,0.941245,...,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0
4,2220,400732.95,45.302280,-124.965970,-28.2990,-0.5122,0.0346,0.2801,-1.819055,0.826606,...,0.4400,0.7283,0.0179,0.0159,0.0160,0.0301,0.0308,0.1465,0e0000d3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865079,2220,469048.15,45.302056,-124.965841,-23.4901,-0.9726,1.0631,-0.1158,2.405454,-1.053228,...,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0
3865080,2220,469048.20,45.302055,-124.965840,-23.4962,-0.9804,1.0885,-0.1416,2.848932,-0.735741,...,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0
3865081,2220,469048.25,45.302055,-124.965839,-23.5037,-0.9709,1.1255,-0.1728,3.109084,-0.440885,...,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0
3865082,2220,469048.30,45.302054,-124.965838,-23.5129,-0.9542,1.1578,-0.2087,3.184167,-0.161808,...,0.3184,0.7405,0.0172,0.0147,0.0206,0.0252,0.0257,0.1263,0e0000d3,0


In [117]:
vel_std_df = vel_std_df.drop_duplicates(subset="time", keep="first")
vel_std_df = vel_std_df.rename(
    columns={
        "ant_e": "vel_e",
        "ant_n": "vel_n",
        "ant_u": "vel_u",
        "ant_e std": "v_sde",
        "ant_n std": "v_sdn",
        "ant_u std": "v_sdu",
        "time":"dts",
    },
    errors="raise"
)
vel_std_df = vel_std_df[
    [
        'dts', 'vel_e', 'vel_n', 'vel_u',
        'v_sde', 'v_sdn', 'v_sdu'
    ]
]
vel_std_df

,dts,vel_e,vel_n,vel_u,v_sde,v_sdn,v_sdu
0,7.122935e+08,0.1014,-0.6858,0.2413,0.0159,0.0179,0.0160
1,7.122935e+08,0.0788,-0.6380,0.2734,0.0159,0.0179,0.0160
2,7.122935e+08,0.0534,-0.5833,0.2926,0.0159,0.0179,0.0160
3,7.122935e+08,0.0405,-0.5440,0.2950,0.0159,0.0179,0.0160
4,7.122935e+08,0.0346,-0.5122,0.2801,0.0159,0.0179,0.0160
...,...,...,...,...,...,...,...
3865079,7.123618e+08,1.0631,-0.9726,-0.1158,0.0147,0.0172,0.0206
3865080,7.123618e+08,1.0885,-0.9804,-0.1416,0.0147,0.0172,0.0206
3865081,7.123618e+08,1.1255,-0.9709,-0.1728,0.0147,0.0172,0.0206
3865082,7.123618e+08,1.1578,-0.9542,-0.2087,0.0147,0.0172,0.0206


In [118]:
"""
def load_gps_solutions(
    files: List[str], time_round: int = constants.DELAY_TIME_PRECISION
) -> pd.DataFrame:
"""
from gnatss.loaders import load_gps_solutions
gps_df = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/posfilter/POS_FREED_TRANS_TWTT',
        'tests/data/2022/NCL1/209/posfilter/POS_FREED_TRANS_TWTT',
        'tests/data/2022/NCL1/210/posfilter/POS_FREED_TRANS_TWTT',

    ],
)
gps_df = gps_df.rename(
    columns={
        "time":"dts",
        "xx": "sdx",
        "yy": "sdy",
        "zz": "sdz",
    },
    errors="raise"
)

gps_df = gps_df[
    [
        'dts', 'x', 'y', 'z', 'sdx', 'sdy', 'sdz'
    ]
]

In [119]:
gps_df

,dts,x,y,z,sdx,sdy,sdz
0,7.122155e+08,-2575288.225,-3682570.227,4511064.310,0.000764,0.001004,0.000887
1,7.122155e+08,-2575286.789,-3682571.528,4511065.018,0.000315,0.000555,0.000438
2,7.122155e+08,-2575286.735,-3682571.582,4511065.013,0.000413,0.000652,0.000536
3,7.122155e+08,-2575286.727,-3682571.590,4511065.010,0.000476,0.000715,0.000599
4,7.122155e+08,-2575279.439,-3682576.054,4511065.580,0.000763,0.001003,0.000887
...,...,...,...,...,...,...,...
47984,7.123964e+08,-2575338.610,-3682612.733,4511002.670,0.000504,0.000542,0.000636
47985,7.123964e+08,-2575336.177,-3682621.936,4510996.736,0.000761,0.000800,0.000894
47986,7.123964e+08,-2575336.427,-3682622.326,4510995.568,0.000312,0.000351,0.000445
47987,7.123964e+08,-2575336.394,-3682622.347,4510995.578,0.000379,0.000418,0.000511


In [120]:
all_df = gps_df.merge(vel_std_df, on="dts", how="inner")

In [121]:
all_df

,dts,x,y,z,sdx,sdy,sdz,vel_e,vel_n,vel_u,v_sde,v_sdn,v_sdu
0,712215465.0,-2575288.225,-3682570.227,4511064.310,0.000764,0.001004,0.000887,0.9704,0.4581,0.3102,0.0150,0.0172,0.0195
1,712215480.0,-2575279.439,-3682576.054,4511065.580,0.000763,0.001003,0.000887,0.6910,0.1621,-0.4804,0.0143,0.0165,0.0194
2,712215495.0,-2575269.558,-3682579.137,4511068.196,0.000764,0.001003,0.000887,0.4897,0.0183,0.0954,0.0146,0.0168,0.0193
3,712215510.0,-2575262.122,-3682585.470,4511066.817,0.000762,0.001003,0.000885,0.5685,-0.3917,-0.5390,0.0147,0.0170,0.0190
4,712215525.0,-2575253.070,-3682595.498,4511065.769,0.000764,0.001007,0.000886,1.4913,-1.1755,0.4341,0.0155,0.0180,0.0198
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16343,712396326.0,-2575309.468,-3682598.666,4511031.049,0.000761,0.000800,0.000893,-0.4071,-0.5584,0.1908,0.0136,0.0171,0.0196
16344,712396341.0,-2575323.088,-3682597.394,4511023.818,0.000762,0.000801,0.000894,-0.6366,-0.3902,-0.6910,0.0141,0.0176,0.0209
16345,712396356.0,-2575333.192,-3682602.175,4511014.150,0.000761,0.000800,0.000893,0.0372,-0.6670,-0.1427,0.0136,0.0172,0.0198
16346,712396371.0,-2575338.200,-3682610.917,4511003.893,0.000761,0.000800,0.000893,-0.3284,-0.7742,0.3550,0.0137,0.0174,0.0196


In [122]:
all_df.columns

Index(['dts', 'x', 'y', 'z', 'sdx', 'sdy', 'sdz', 'vel_e', 'vel_n', 'vel_u',
       'v_sde', 'v_sdn', 'v_sdu'],
      dtype='object')

In [123]:
all_df["sdxy"] = np.sqrt(all_df.sdx * all_df.sdy)
all_df["sdxz"] = np.sqrt(all_df.sdx * all_df.sdz)
all_df["sdyz"] = np.sqrt(all_df.sdy * all_df.sdz)
all_df["v_sden"] = np.sqrt(all_df.v_sde * all_df.v_sdn)
all_df["v_sdeu"] = np.sqrt(all_df.v_sde * all_df.v_sdu)
all_df["v_sdnu"] = np.sqrt(all_df.v_sdn * all_df.v_sdu)

all_df = all_df.drop_duplicates(subset="dts", keep="first")

all_df.reset_index(drop=True)
all_df.sort_values("dts").reset_index(drop=True)
first_pos = all_df[~all_df.x.isnull()].iloc[0].name
all_df = all_df.loc[first_pos:].reset_index(drop=True)
df = all_df.to_records()
df

rec.array([(    0, 7.12215465e+08, -2575288.225, -3682570.227, 4511064.31 , 0.00076427, 0.00100377, 0.00088748,  0.9704,  0.4581,  0.3102, 0.015 , 0.0172, 0.0195, 0.00087587, 0.00082357, 0.00094384, 0.01606238, 0.01710263, 0.01831393),
           (    1, 7.12215480e+08, -2575279.439, -3682576.054, 4511065.58 , 0.00076328, 0.0010029 , 0.00088686,  0.691 ,  0.1621, -0.4804, 0.0143, 0.0165, 0.0194, 0.00087493, 0.00082276, 0.0009431 , 0.01536066, 0.01665593, 0.01789134),
           (    2, 7.12215495e+08, -2575269.558, -3682579.137, 4511068.196, 0.00076364, 0.00100313, 0.00088694,  0.4897,  0.0183,  0.0954, 0.0146, 0.0168, 0.0193, 0.00087523, 0.00082298, 0.00094325, 0.01566142, 0.0167863 , 0.01800667),
           ...,
           (12000, 7.12396356e+08, -2575333.192, -3682602.175, 4511014.15 , 0.0007609 , 0.00079978, 0.00089304,  0.0372, -0.667 , -0.1427, 0.0136, 0.0172, 0.0198, 0.0007801 , 0.00082433, 0.00084513, 0.01529444, 0.01640975, 0.01845427),
           (12001, 7.12396371e+08, -2575

In [124]:
x, P, K, Pp = run_filter_simulation(df)

12003/12003
Smoothing results
12003/12003


In [125]:
x

array([[[-2.57528893e+06],
        [-3.68257198e+06],
        [ 4.51106344e+06],
        [ 6.32687758e-01],
        [-2.71836093e-01],
        [ 1.42587390e-01]],

       [[-2.57527944e+06],
        [-3.68257605e+06],
        [ 4.51106558e+06],
        [ 6.58526280e-01],
        [-2.05772402e-01],
        [ 1.74151475e-01]],

       [[-2.57526956e+06],
        [-3.68257914e+06],
        [ 4.51106820e+06],
        [ 4.95981975e-01],
        [-4.22238883e-01],
        [-9.17056919e-02]],

       ...,

       [[-2.57533319e+06],
        [-3.68260218e+06],
        [ 4.51101415e+06],
        [-3.33766779e-01],
        [-5.82740876e-01],
        [-6.83597141e-01]],

       [[-2.57533820e+06],
        [-3.68261092e+06],
        [ 4.51100389e+06],
        [ 1.34547614e-01],
        [-7.34518835e-01],
        [-4.77428033e-01]],

       [[-2.57533618e+06],
        [-3.68262194e+06],
        [ 4.51099674e+06],
        [ 1.20679626e-01],
        [-7.57060610e-01],
        [-5.26737759e-01]]])

In [126]:
P

array([[[ 3.59809468e-01,  1.56596311e-06,  1.36727617e-06,
         -2.39672423e-02, -3.92130995e-08, -3.35090373e-08],
        [ 1.56596311e-06,  3.59810337e-01,  1.82586683e-06,
         -3.92082913e-08, -2.39672641e-02, -4.59809627e-08],
        [ 1.36727616e-06,  1.82586684e-06,  3.59809868e-01,
         -3.35097116e-08, -4.59874354e-08, -2.39672516e-02],
        [-2.39672423e-02, -3.92082914e-08, -3.35097116e-08,
          1.59856441e-03,  1.66636869e-09,  1.39606124e-09],
        [-3.92130995e-08, -2.39672641e-02, -4.59874354e-08,
          1.66636869e-09,  1.59856534e-03,  1.96454501e-09],
        [-3.35090371e-08, -4.59809630e-08, -2.39672516e-02,
          1.39606124e-09,  1.96454501e-09,  1.59856477e-03]],

       [[ 5.82594811e-07,  7.65492238e-07,  6.76919606e-07,
         -3.87846109e-08, -5.09680197e-08, -4.50629632e-08],
        [ 7.65492238e-07,  1.00580776e-06,  8.89428986e-07,
         -5.09604927e-08, -6.69687126e-08, -5.92098461e-08],
        [ 6.76919606e-07,  8.8

In [127]:
K

array([[[ 6.40382792e-01,  0.00000000e+00,  0.00000000e+00,
         -9.58178742e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  6.40382792e-01,  0.00000000e+00,
          0.00000000e+00, -9.58178742e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  6.40382792e-01,
          0.00000000e+00,  0.00000000e+00, -9.58178742e+00],
        [ 2.39544685e-02,  0.00000000e+00,  0.00000000e+00,
          6.39085259e-01,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  2.39544685e-02,  0.00000000e+00,
          0.00000000e+00,  6.39085259e-01,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  2.39544685e-02,
          0.00000000e+00,  0.00000000e+00,  6.39085259e-01]],

       [[ 1.74851947e-06,  2.05658015e-06,  2.06540058e-06,
         -1.55140051e-05, -2.03874190e-05, -1.80253719e-05],
        [ 2.29744250e-06,  2.70221456e-06,  2.71380401e-06,
         -2.03844082e-05, -2.67877625e-05, -2.36841837e-05],
        [ 2.03161286e-06,  2.3

In [128]:
Pp

array([[[ 2.26000469e+02,  0.00000000e+00,  0.00000000e+00,
          1.50000000e+01,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  2.26000469e+02,  0.00000000e+00,
          0.00000000e+00,  1.50000000e+01,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  2.26000469e+02,
          0.00000000e+00,  0.00000000e+00,  1.50000000e+01],
        [ 1.50000000e+01,  0.00000000e+00,  0.00000000e+00,
          1.00250000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.50000000e+01,  0.00000000e+00,
          0.00000000e+00,  1.00250000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.50000000e+01,
          0.00000000e+00,  0.00000000e+00,  1.00250000e+00]],

       [[ 2.37772741e+00,  7.04801033e-02, -5.70759735e-01,
          1.58483827e-01,  4.69856362e-03, -3.80507462e-02],
        [ 7.04801033e-02,  2.57994257e+00,  4.70145120e-01,
          4.69852313e-03,  1.71964724e-01,  3.13428332e-02],
        [-5.70759735e-01,  4.7